# Marketing G2

In [32]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact 

## Import data sets

In [4]:
movies=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\movies_full')
ratings=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\ratings')
users=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\users')

## EDA

In [4]:
n_users = ratings.userId.unique().shape[0]
n_items = ratings.movieId.unique().shape[0]
print (str(n_users) + ' users')
print (str(n_items) + ' items')

610 users
9724 items


In [8]:
px.histogram(ratings.rating)
#Los usuarios no les tiran tan duro a las peliculas#

In [16]:
ratings.groupby(['rating'])['userId'].count()

rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: userId, dtype: int64

In [18]:
URmatrix = pd.pivot_table(ratings, values='rating', index='userId', columns='movieId').fillna(0)
URmatrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
ratings = URmatrix.values
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))#!!!!!!!!!!!!! Mucha Sparsity; Muchos 0s

Sparsity: 1.70%


# Recomendatios Systems

## Popularity Based

In [47]:
# Train-Test Split
ratings_train, ratings_test = train_test_split(ratings, test_size = 0.4, random_state=42)
print(ratings_train.shape)
print(ratings_test.shape)

(60501, 4)
(40335, 4)


In [7]:
def popularity(train,movieId,userId):
    train_data_grouped=train.groupby([movieId])[userId].count().reset_index()
    train_data_grouped.rename(columns={userId:'score'},inplace=True)
    train_data_sort=train_data_grouped.sort_values(['score',movieId],ascending=[0,1])
    train_data_sort['Rank']=train_data_sort['score'].rank(ascending=0,method='first')
    popularity_recommendations=train_data_sort.head(10)
    return popularity_recommendations

In [18]:
pop_rec_sys=popularity(ratings,'movieId','userId')

In [19]:
pop_rec_sys

,movieId,score,Rank
314,356,329,1.0
277,318,317,2.0
257,296,307,3.0
510,593,279,4.0
1938,2571,278,5.0
224,260,251,6.0
418,480,238,7.0
97,110,237,8.0
507,589,224,9.0
461,527,220,10.0


Como el sistema de recomendacion basado en popularidad es parcialmente inmutable, solo se obtiene un resultado y no un modelo predictivo, como tal.

In [21]:
pop_rec_sys_final=pd.merge(pop_rec_sys,movies[['movieId','title']],on='movieId', how='left')
pop_rec_sys_final

,movieId,score,Rank,title
0,356,329,1.0,Forrest Gump (1994)
1,318,317,2.0,"Shawshank Redemption, The (1994)"
2,296,307,3.0,Pulp Fiction (1994)
3,593,279,4.0,"Silence of the Lambs, The (1991)"
4,2571,278,5.0,"Matrix, The (1999)"
5,260,251,6.0,Star Wars: Episode IV - A New Hope (1977)
6,480,238,7.0,Jurassic Park (1993)
7,110,237,8.0,Braveheart (1995)
8,589,224,9.0,Terminator 2: Judgment Day (1991)
9,527,220,10.0,Schindler's List (1993)


Shotout to Rogish Ghosh to help us with the popularity based recommendation system.

## Content Based

In [24]:
movies.head(3)
movies_predictive=movies.drop(columns='genres')

In [27]:
movies_predictive

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),False,False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji (1995),False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men (1995),False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,4,Waiting to Exhale (1995),False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4,5,Father of the Bride Part II (1995),False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),False,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
9738,193583,No Game No Life: Zero (2017),False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
9739,193585,Flint (2017),False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9740,193587,Bungo Stray Dogs: Dead Apple (2018),False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [25]:
ratings

,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 13:45:03
1,1,3,4.0,2000-07-30 13:20:47
2,1,6,4.0,2000-07-30 13:37:04
3,1,47,5.0,2000-07-30 14:03:35
4,1,50,5.0,2000-07-30 13:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 16:53:22
100832,610,168248,5.0,2017-05-03 17:21:31
100833,610,168250,5.0,2017-05-08 14:50:47
100834,610,168252,5.0,2017-05-03 16:19:12


In [30]:
#def recomendar(user_id=list(users['userId'].value_counts().index)):
    
    #ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movieId'].to_numpy()
    movies_dum=movies[['movieId','title']]
    movies_r=movies_dum[movies_dum['movieId'].isin(l_movies_r)]
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=movies_r.groupby("indice").mean()
    
    
    movies_nr=movies_dum[~movies_dum['movieId'].isin(l_movies_r)]
    movies_nr=movies_nr.drop(columns=['movieId','title'])
    model=sk.neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)
    
    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]
    leidos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]
    
    return recomend_b


#recomendar(610)



In [55]:
movies.head(1)

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,False,False,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [56]:
movies_dum1=movies[['movieId','title','genres']]
movies_dum1['title'].nunique()
movies_dum1['genres'].nunique()

col_dum=['title','genres']
movies_dum2=pd.get_dummies(movies_dum1,columns=col_dum)
movies_dum2.shape

(9742, 10689)

In [57]:
l_movies_r=ratings_train['movieId'].to_numpy()
movies_dum2[['movieId','title']]=ratings_test[['movieId','title']]
movies_r=movies_dum2[movies_dum2['movieId'].isin(l_movies_r)]
movies_r=movies_r.drop(columns=['movieId'])
movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
centroide=movies_r.groupby("indice").mean()
    
    
movies_nr=movies_dum2[~movies_dum2['movieId'].isin(l_movies_r)]
movies_nr=movies_nr.drop(columns=['movieId'])
model=sk.neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(movies_nr)
dist, idlist = model.kneighbors(centroide)
    
#ids=idlist[0]
#recomend_b=movies.loc[ids][['title','movieId']]
#leidos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]
    
#recomend_b

KeyError: "['title'] not in index"

In [53]:
centroide


""
indice
1


In [36]:
l_movies_r

array([     1,      3,      6, ..., 168250, 168252, 170875], dtype=int64)

In [33]:
print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x0000024341D79C10>
